In [1]:
import pandas as pd
import json
from normaliza_iniciativas.normalizer import clear_string

Abrindo arquivo XLS do formulário

In [2]:
survey_file = "data/cadastro_de_iniciativas_mapa_das_periferias_atUcxaKhdkAJsHhiPzXJ2E_configs.xls"
survey = pd.read_excel(survey_file, sheet_name="survey")

In [3]:
# normalizando os nomes das colunas
survey.columns = [clear_string(col, "_") for col in survey.columns]

display(survey.head(10))

,type,name,label_portugues_brasil_pt,hint_portugues_brasil_pt,required,appearance,relevant,file,choice_filter,parameters
0,start,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,end,end,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,begin_group,apresentacao,NaN,NaN,False,field-list,NaN,NaN,NaN,NaN
3,hidden,id_usuario,id_usuario,NaN,False,NaN,NaN,NaN,NaN,NaN
4,note,dica_apresentacao,Este formulário possui 19 perguntas e o preenc...,NaN,False,NaN,NaN,NaN,NaN,NaN
5,end_group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,begin_group,grupo_dados_iniciativa,**Dados da Iniciativa**,NaN,False,field-list,NaN,NaN,NaN,NaN
7,select_one zs1dz05,relacao_com_iniciativa,1. Qual a sua relação com a iniciativa que est...,NaN,True,minimal,NaN,NaN,NaN,NaN
8,text,relacao_com_iniciativa_outra,Escreva qual a sua relação com a iniciativa qu...,NaN,True,NaN,${relacao_com_iniciativa} = 'outro',NaN,NaN,NaN
9,text,nome_iniciativa,2. Qual é o nome da iniciativa?,NaN,True,NaN,NaN,NaN,NaN,NaN


In [21]:
selections_questions = survey[survey["type"].str.startswith("select")]
display(selections_questions.head(10))

,type,name,label_portugues_brasil_pt,hint_portugues_brasil_pt,required,appearance,relevant,file,choice_filter,parameters
7,select_one zs1dz05,relacao_com_iniciativa,1. Qual a sua relação com a iniciativa que est...,NaN,True,minimal,NaN,NaN,NaN,NaN
10,select_one Estado,estado_iniciativa,3. Em qual estado a iniciativa atua?,NaN,True,minimal,NaN,NaN,NaN,NaN
11,select_one_from_file municipio.csv,municipio_iniciativa,4. Qual é o município em que sua iniciativa atua?,NaN,True,minimal,${estado_iniciativa} != '53',NaN,filter=${estado_iniciativa},NaN
12,select_one_from_file regiao.csv,regiao_iniciativa,4. Qual é a região administrativa em que sua i...,NaN,True,minimal,${estado_iniciativa} = '53',NaN,filter=${estado_iniciativa},NaN
13,select_one jx0jm13,possui_endereco_completo_inici,5. A iniciativa que está sendo cadastrada poss...,NaN,True,NaN,NaN,NaN,NaN,NaN
14,select_one fi3ig04,logradouro_iniciativa,Qual o tipo de logradouro?,Logradouro é o termo utilizado pelos Correios ...,False,minimal,NaN,NaN,NaN,NaN
16,select_one ex6vi32,sn_iniciativa,"Caso seu logradouro não tenha número, selecion...",NaN,False,NaN,NaN,NaN,NaN,NaN
23,select_multiple fj6yn86,onde_como_iniciativa,7. Onde e como acontece a iniciativa nas comun...,NaN,True,NaN,NaN,NaN,NaN,NaN
25,select_multiple bx78r45,rede_social_site_iniciativa,8. A iniciativa possui redes sociais ou site d...,Cole nos campos o(s) link(s).,True,NaN,NaN,NaN,NaN,NaN
34,select_one fh9pk26,responsaveis_iniciativa,9. Os responsáveis pela iniciativa são pessoas...,NaN,True,minimal,NaN,NaN,NaN,NaN


Abrindo respostas padrão

In [19]:
standard_answers = pd.read_excel(survey_file, sheet_name="choices")
standard_answers.columns = [clear_string(col, "_") for col in standard_answers.columns]
display(standard_answers.head(10))

,list_name,name,label_portugues_brasil_pt,order
0,zs1dz05,lideran_a_respons_vel,Sou liderança/responsável,NaN
1,zs1dz05,membro,Sou membro,NaN
2,zs1dz05,parceiro_apoiador,Sou parceiro/apoiador,NaN
3,zs1dz05,outro,Outra,NaN
4,Estado,12,Acre,NaN
5,Estado,27,Alagoas,NaN
6,Estado,16,Amapá,NaN
7,Estado,13,Amazonas,NaN
8,Estado,29,Bahia,NaN
9,Estado,23,Ceará,NaN


Pega apenas os grupos de perguntas

In [4]:
groups = survey[survey["type"] == "begin_group"].copy()
# pega o index inicial do grupo e coloca na coluna "begin_group_id"
groups["begin_group_id"] = groups.index + 1

# pega o index final do grupo e coloca na coluna "end_group_id" utilizando o index do próximo grupo
groups["end_group_id"] = groups["begin_group_id"].shift(-1) - 1
groups["end_group_id"] = groups["end_group_id"].fillna(len(survey) - 1).astype(int)

display(groups)
print(f"Quantidade de Grupos: {len(groups)}")

,type,name,label_portugues_brasil_pt,hint_portugues_brasil_pt,required,appearance,relevant,file,choice_filter,parameters,begin_group_id,end_group_id
2,begin_group,apresentacao,NaN,NaN,False,field-list,NaN,NaN,NaN,NaN,3,6
6,begin_group,grupo_dados_iniciativa,**Dados da Iniciativa**,NaN,False,field-list,NaN,NaN,NaN,NaN,7,33
33,begin_group,grupo_organizacao,Conte mais sobre como a iniciativa se organiza.,NaN,False,field-list,NaN,NaN,NaN,NaN,34,45
45,begin_group,grupo_temas,NaN,NaN,False,field-list,NaN,NaN,NaN,NaN,46,53
53,begin_group,grupo_imagens,Envie algumas fotos que demonstrem as ações e ...,NaN,False,field-list,NaN,NaN,NaN,NaN,54,65
65,begin_group,grupo_autorizacoes,Autorizações,NaN,False,field-list,NaN,NaN,NaN,NaN,66,68


Quantidade de Grupos: 6


Pegando as colunas do formulário

In [5]:
# Todas as linhas onde type não termina com group
questions = survey[~survey["type"].str.endswith("group")].copy()
display(questions.head(10))

,type,name,label_portugues_brasil_pt,hint_portugues_brasil_pt,required,appearance,relevant,file,choice_filter,parameters
0,start,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,end,end,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hidden,id_usuario,id_usuario,NaN,False,NaN,NaN,NaN,NaN,NaN
4,note,dica_apresentacao,Este formulário possui 19 perguntas e o preenc...,NaN,False,NaN,NaN,NaN,NaN,NaN
7,select_one zs1dz05,relacao_com_iniciativa,1. Qual a sua relação com a iniciativa que est...,NaN,True,minimal,NaN,NaN,NaN,NaN
8,text,relacao_com_iniciativa_outra,Escreva qual a sua relação com a iniciativa qu...,NaN,True,NaN,${relacao_com_iniciativa} = 'outro',NaN,NaN,NaN
9,text,nome_iniciativa,2. Qual é o nome da iniciativa?,NaN,True,NaN,NaN,NaN,NaN,NaN
10,select_one Estado,estado_iniciativa,3. Em qual estado a iniciativa atua?,NaN,True,minimal,NaN,NaN,NaN,NaN
11,select_one_from_file municipio.csv,municipio_iniciativa,4. Qual é o município em que sua iniciativa atua?,NaN,True,minimal,${estado_iniciativa} != '53',NaN,filter=${estado_iniciativa},NaN
12,select_one_from_file regiao.csv,regiao_iniciativa,4. Qual é a região administrativa em que sua i...,NaN,True,minimal,${estado_iniciativa} = '53',NaN,filter=${estado_iniciativa},NaN


In [ ]:
# Adiciona o grupo a cada pergunta
questions["group"] = None
for i, group in groups.iterrows():
    questions.loc[group["begin_group_id"] : group["end_group_id"], "group"] = group[
        "name"
    ]

# Adiciona a coluna choices_name a cada pergunta
questions["choices_name"] = None
# adiciona o valor de choices_name a cada pergunta presente na coluna type da pergunda
questions["choices_name"] = questions["type"].apply(
    lambda x: (
        x.split(" ")[1]
        if x.split(" ")[0] in ["select_one", "select_multiple"]
        else None
    )
)

display(questions.head(10))

,type,name,label_portugues_brasil_pt,hint_portugues_brasil_pt,required,appearance,relevant,file,choice_filter,parameters,group,choices_name
0,start,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,end,end,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,hidden,id_usuario,id_usuario,NaN,False,NaN,NaN,NaN,NaN,NaN,apresentacao,None
4,note,dica_apresentacao,Este formulário possui 19 perguntas e o preenc...,NaN,False,NaN,NaN,NaN,NaN,NaN,apresentacao,None
7,select_one zs1dz05,relacao_com_iniciativa,1. Qual a sua relação com a iniciativa que est...,NaN,True,minimal,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,zs1dz05
8,text,relacao_com_iniciativa_outra,Escreva qual a sua relação com a iniciativa qu...,NaN,True,NaN,${relacao_com_iniciativa} = 'outro',NaN,NaN,NaN,grupo_dados_iniciativa,None
9,text,nome_iniciativa,2. Qual é o nome da iniciativa?,NaN,True,NaN,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,None
10,select_one Estado,estado_iniciativa,3. Em qual estado a iniciativa atua?,NaN,True,minimal,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,Estado
11,select_one_from_file municipio.csv,municipio_iniciativa,4. Qual é o município em que sua iniciativa atua?,NaN,True,minimal,${estado_iniciativa} != '53',NaN,filter=${estado_iniciativa},NaN,grupo_dados_iniciativa,None
12,select_one_from_file regiao.csv,regiao_iniciativa,4. Qual é a região administrativa em que sua i...,NaN,True,minimal,${estado_iniciativa} = '53',NaN,filter=${estado_iniciativa},NaN,grupo_dados_iniciativa,None


Pegando apenas as colunas que possui choices

In [41]:
choices_questions = questions[questions["choices_name"].notnull()].copy()
display(choices_questions.head(10))

,type,name,label_portugues_brasil_pt,hint_portugues_brasil_pt,required,appearance,relevant,file,choice_filter,parameters,group,choices_name
7,select_one zs1dz05,relacao_com_iniciativa,1. Qual a sua relação com a iniciativa que est...,NaN,True,minimal,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,zs1dz05
10,select_one Estado,estado_iniciativa,3. Em qual estado a iniciativa atua?,NaN,True,minimal,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,Estado
13,select_one jx0jm13,possui_endereco_completo_inici,5. A iniciativa que está sendo cadastrada poss...,NaN,True,NaN,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,jx0jm13
14,select_one fi3ig04,logradouro_iniciativa,Qual o tipo de logradouro?,Logradouro é o termo utilizado pelos Correios ...,False,minimal,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,fi3ig04
16,select_one ex6vi32,sn_iniciativa,"Caso seu logradouro não tenha número, selecion...",NaN,False,NaN,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,ex6vi32
23,select_multiple fj6yn86,onde_como_iniciativa,7. Onde e como acontece a iniciativa nas comun...,NaN,True,NaN,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,fj6yn86
25,select_multiple bx78r45,rede_social_site_iniciativa,8. A iniciativa possui redes sociais ou site d...,Cole nos campos o(s) link(s).,True,NaN,NaN,NaN,NaN,NaN,grupo_dados_iniciativa,bx78r45
34,select_one fh9pk26,responsaveis_iniciativa,9. Os responsáveis pela iniciativa são pessoas...,NaN,True,minimal,NaN,NaN,NaN,NaN,grupo_organizacao,fh9pk26
35,select_multiple ny0jq61,definicao_responsaveis_iniciat,10. Como você define o(s) grupo(s) e/ou instit...,Marque quantas opções forem necessárias.,True,NaN,NaN,NaN,NaN,NaN,grupo_organizacao,ny0jq61
37,select_one jj8ft91,possui_cnpj_iniciativa,11. A organização/coletivo possui CNPJ?,NaN,True,minimal,NaN,NaN,NaN,NaN,grupo_organizacao,jj8ft91


Pegando apenas colunas que possuem a opção "Outro"

In [56]:
# pega apenas as questões que possuem "outro" no type

other_questions = questions[questions["name"].str.endswith(("outra", "outro"))].copy()

display(other_questions.head(10))

,type,name,label_portugues_brasil_pt,hint_portugues_brasil_pt,required,appearance,relevant,file,choice_filter,parameters,group,choices_name
8,text,relacao_com_iniciativa_outra,Escreva qual a sua relação com a iniciativa qu...,NaN,True,NaN,${relacao_com_iniciativa} = 'outro',NaN,NaN,NaN,grupo_dados_iniciativa,None


Abrindo resposta dos formulários

In [38]:
survey_data_file = (
    "data/cadastro_de_iniciativas_mapa_das_periferias_atUcxaKhdkAJsHhiPzXJ2E_data.json"
)


survey_data = json.loads(open(survey_data_file).read())
# Abre apenas a chave results do arquivo JSON
survey_data = pd.DataFrame(survey_data["results"])
# survey_data.columns = [clear_string(col, "_") for col in survey_data.columns]
# Colunas de uso interno do Kobo
ignore_cols = [
    "_id",
    "formhub/uuid",
    "start",
    "end",
    "__version__",
    "meta/instanceID",
    "_xform_id_string",
    "_uuid",
    "_attachments",
    "_status",
    "_geolocation",
    "_submission_time",
    "_tags",
    "_notes",
    "_validation_status",
    "_submitted_by",
]
survey_data_cols = {col: col.split("/")[-1] for col in survey_data.columns if col not in ignore_cols}
# ordena survey_data_cols pela chave
# survey_data_cols = dict(sorted(survey_data_cols.items(), key=lambda item: item[0]))

survey_data = survey_data.rename(columns=survey_data_cols)


In [37]:
display(survey_data_cols)

{'grupo_dados_iniciativa/relacao_com_iniciativa': 'relacao_com_iniciativa',
 'grupo_dados_iniciativa/nome_iniciativa': 'nome_iniciativa',
 'grupo_dados_iniciativa/estado_iniciativa': 'estado_iniciativa',
 'grupo_dados_iniciativa/municipio_iniciativa': 'municipio_iniciativa',
 'grupo_dados_iniciativa/possui_endereco_completo_inici': 'possui_endereco_completo_inici',
 'grupo_dados_iniciativa/logradouro_iniciativa': 'logradouro_iniciativa',
 'grupo_dados_iniciativa/nome_logradouro_iniciativa': 'nome_logradouro_iniciativa',
 'grupo_dados_iniciativa/sn_iniciativa': 'sn_iniciativa',
 'grupo_dados_iniciativa/complemento_iniciativa': 'complemento_iniciativa',
 'grupo_dados_iniciativa/bairro_comunidade_iniciativa': 'bairro_comunidade_iniciativa',
 'grupo_dados_iniciativa/cep_iniciativa': 'cep_iniciativa',
 'grupo_dados_iniciativa/outros_bairros_comunidades_ini': 'outros_bairros_comunidades_ini',
 'grupo_dados_iniciativa/onde_como_iniciativa': 'onde_como_iniciativa',
 'grupo_dados_iniciativa/red

In [39]:
display(survey_data.dtypes)
rel_iniciativa = survey_data[
    ~survey_data["relacao_com_iniciativa_outra"].isnull()
]

rel_iniciativa = rel_iniciativa[
    [
        "_id",
        "formhub/uuid",
        "relacao_com_iniciativa",
        "relacao_com_iniciativa_outra",
    ]
]
display(rel_iniciativa.head(2))

_id                                int64
formhub/uuid                      object
start                             object
end                               object
relacao_com_iniciativa            object
                                   ...  
regiao_iniciativa                 object
outros_equipes_populacoes_inic    object
outra_rede_social_iniciativa      object
id_usuario                        object
relacao_com_iniciativa_outra      object
Length: 69, dtype: object

,_id,formhub/uuid,relacao_com_iniciativa,relacao_com_iniciativa_outra
215,639250641,81c5007f7ab541d98a5b439029a8b6a5,outro,Sou educomunicador dos jovens do coletivo
